# Week 7

In [6]:
%pip install surprise

  Using cached surprise-0.1-py2.py3-none-any.whl.metadata (327 bytes)
  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Compiling surprise/similarities.pyx because it changed.
      Compiling surprise/prediction_algorithms/matrix_factorization.pyx because it changed.
      Compiling surprise/prediction_algorithms/optimize_baselines.pyx because it changed.
      Compiling surprise/prediction_algorithms/slope_one.pyx because it changed.
      Compiling surprise/prediction_algorithms/co_clustering.pyx because it changed.
      [1/5] Cythonizing surprise/prediction_algorithms/co_clustering.pyx
      
      Error compiling Cython file:
      ------------------------------------------------------------
      ...
              self.avg_cltr_i = avg_cltr_i
              self.avg_cocltr = avg_cocltr
      
              return self
      
          def compute_averages(self, np.ndarray[np.int_t] cltr_u,
                                                   ^
  

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np  


In [8]:
test_df = pd.read_parquet('test_video_games.parquet')
train_df = pd.read_parquet('train_video_games.parquet')
items_highly_rated_freq = pd.read_csv('item_highly_rated_freq.csv')

Based on the frequency of the most rated items computed in Week 6,
implement the TopPop Recommender System, which always recommends
the same top-k items sorted decreasingly by the number of “high” ratings
(e.g., → 3) in the training split (train video games.parquet).

In [9]:
# implement based line TopPop recommender system
top_pop_items = items_highly_rated_freq.head(10)
# just recommend the top 10 most popular highly rated (defined as having a rating >= 3) items
print(top_pop_items)


      item_id  highly_rated_count  average_rating_by_item
0  B0086VPUHI                 154                4.425150
1  B00BN5T30E                 150                4.428571
2  B07YBXFDYN                 146                4.329268
3  B00BGA9WK2                 136                4.432432
4  B007CM0K86                 123                4.554688
5  B00KIWEMIG                 109                4.237705
6  B07YBWT3PK                 104                4.107438
7  B07YBXFF99                 103                4.345455
8  B014R4KYMS                 102                4.532110
9  B004HD55VK                 102                4.615385


Choose at least one neighborhood-based model and one latent factor model (both type of colaborative filtering)
that uses the observed user-item ratings in the training set to predict the
unobserved ratings. Report your choice of models.   

Does unobserved mean empty cell? There is no missing data as seen in from the data exploration

Use 5-fold cross-validation on the training set to tune the hyperparameters
of the chosen models (similarity measure and number of neighbors for the
neighborhood-based model; number of latent factors and number of epochs
for the latent factor model).

Choose an evaluation measure that is suitable for this task and justify your
motivation in using it. Report the optimal hyperparameters together with
the scores of your chosen measure, averaged over the 5 folds.
• Run the models with the optimal hyperparameters to the whole training
set

In [10]:
# convert train and test to csv files
train_df.to_csv('train_video_games.csv', index=False)
test_df.to_csv('test_video_games.csv', index=False) 

### Item Based KNN Colaborative Filtering




In [ ]:
# create item based KNN colaborative filtering recommender system using the surprise library
# use 5-fold cross validation 
# use Root Mean Square Error (RMSE) as the evaluation metric
from surprise import Dataset, Reader
from surprise import KNNBasic, KNNBaseline
from surprise.model_selection import cross_validate, GridSearchCV

# load the data into surprise format
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train_df[['user_id', 'item_id', 'rating']], reader)



In [ ]:
# train/evaluate the algorithm using grind search 5 fold cross validation
params = {'num_neighbors':[5, 10, 20], 'epochs':[5, 10, 15], 'sim_options': {'name': ['cosine', 'pearson'], 'user_based': [False]}}
algo = KNNBaseline()
cv_results = GridSearchCV(algo, params, cv=5)

# print the average RMSE
print(f"Average RMSE: {np.mean(cv_results['test_rmse'])}")
# np mean is root mean square error?


### SVD 
SVD is a matrix factorization technique that decomposes the user-item interaction matrix into latent factors.


In [ ]:
# train a SVD based collabortative filtering recommender system using the surprise library
from surprise import SVD

algoSVD = SVD()

# data and reader is the same 
# hyperparameters for SVD can be tuned using grid search
params_svd = {'n_factors': [50, 100, 150], 'n_epochs': [20, 30, 40], 'lr_all': [0.002, 0.005, 0.01], 'reg_all': [0.02, 0.05, 0.1]}
cv_results_svd = GridSearchCV(SVD, params_svd, cv=5)
cv_results_svd.fit(data)

# print the average RMSE
print(f"Average RMSE for SVD: {np.mean(cv_results_svd['test_rmse'])}")  
